In [ ]:
class Config:
    name = "baseline-001"
    only_inference = True

    model_name = "../input/roberta-base"
    learning_rate = 1e-5
    max_length = 256
    epochs = 8
    batch_size = 16

    n_fold = 5
    trn_fold = [0, 1, 2, 3, 4]
    seed = 2022
    target_col = "target"
    debug = False

    # Colab Env
    upload_from_colab = False
    api_path = "/content/drive/MyDrive/kaggle.json"
    drive_path = "/content/drive/MyDrive/CommonLit-Readability-Prize"
    
    # Kaggle Env
    kaggle_dataset_path = "../input/baseline-001"

In [ ]:
import os
import json
import warnings
import shutil
import logging
import joblib
import random
import datetime
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import backend as K

In [ ]:
class Logger:
    """参考) https://github.com/ghmagazine/kagglebook/blob/master/ch04-model-interface/code/util.py"""
    def __init__(self, path):
        self.general_logger = logging.getLogger(path)
        stream_handler = logging.StreamHandler()
        file_general_handler = logging.FileHandler(os.path.join(path, 'Experiment.log'))
        if len(self.general_logger.handlers) == 0:
            self.general_logger.addHandler(stream_handler)
            self.general_logger.addHandler(file_general_handler)
            self.general_logger.setLevel(logging.INFO)

    def info(self, message):
        # display time
        self.general_logger.info('[{}] - {}'.format(self.now_string(), message))

    @staticmethod
    def now_string():
        return str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    

def seed_everything(seed=42):
    """参考) https://qiita.com/kaggle_grandmaster-arai-san/items/d59b2fb7142ec7e270a5"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
COLAB = "google.colab" in sys.modules
if COLAB:
    print("This environment is Google Colab")
    
    # mount
    from google.colab import drive
    if not os.path.isdir("/content/drive"):
        drive.mount('/content/drive') 
	
    # import library
    ! pip install --quiet transformers
    ! pip install --quiet iterative-stratification
    ! pip install --quiet tensorflow-addons

    # use kaggle api (need kaggle token)
    f = open(Config.api_path, 'r')
    json_data = json.load(f) 
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]
    
    # set dirs
    DRIVE = Config.drive_path
    EXP = (Config.name if Config.name is not None 
           else get("http://172.28.0.2:9000/api/sessions").json()[0]["name"][:-6])
    INPUT = os.path.join(DRIVE, "Input")
    OUTPUT = os.path.join(DRIVE, "Output")
    SUBMISSION = os.path.join(DRIVE, "Submission")
    OUTPUT_EXP = os.path.join(OUTPUT, EXP) 
    EXP_MODEL = os.path.join(OUTPUT_EXP, "model")
    EXP_FIG = os.path.join(OUTPUT_EXP, "fig")
    EXP_PREDS = os.path.join(OUTPUT_EXP, "preds")

    # make dirs
    for d in [INPUT, SUBMISSION, EXP_MODEL, EXP_FIG, EXP_PREDS]:
        os.makedirs(d, exist_ok=True)

    if not os.path.isfile(os.path.join(INPUT, "train.csv.zip")):
        # load dataset
        ! kaggle competitions download -c commonlitreadabilityprize -p $INPUT 
    
    # utils
    logger = Logger(OUTPUT_EXP)

else:
    print("This environment is Kaggle Kernel")
    
    # set dirs
    INPUT = "../input/commonlitreadabilityprize"
    EXP, OUTPUT, SUBMISSION = "./", "./", "./"
    EXP_MODEL = os.path.join(EXP, "model")
    EXP_FIG = os.path.join(EXP, "fig")
    EXP_PREDS = os.path.join(EXP, "preds")
    
    # copy dirs
    if Config.kaggle_dataset_path is not None:
        KD_MODEL = os.path.join(Config.kaggle_dataset_path, "model")
        KD_EXP_PREDS = os.path.join(Config.kaggle_dataset_path, "preds")
        shutil.copytree(KD_MODEL, EXP_MODEL)
        shutil.copytree(KD_EXP_PREDS, EXP_PREDS)

    # make dirs
    for d in [EXP_MODEL, EXP_FIG, EXP_PREDS]:
        os.makedirs(d, exist_ok=True)
        
    # utils
    logger = Logger(EXP)

# utils
warnings.filterwarnings("ignore")
sns.set(style='whitegrid')
seed_everything(seed=Config.seed)

# 2nd import
from transformers import AutoTokenizer, TFAutoModel
import tensorflow_addons as tfa

In [ ]:
# load data
train = pd.read_csv(os.path.join(INPUT, "train.csv.zip" if COLAB else "train.csv"))
test = pd.read_csv(os.path.join(INPUT, "test.csv"))
sample_submission = pd.read_csv(os.path.join(INPUT, "sample_submission.csv"))

if Config.debug:
    train = train.sample(100).reset_index(drop=True)

# cv split
train["fold"] = -1
for i_fold, lst in enumerate(
    KFold(
        n_splits=Config.n_fold, 
        shuffle=True,
        random_state=Config.seed).split(
            X=train, 
            y=train[Config.target_col]
            )):
    
    if i_fold in Config.trn_fold:
        train.loc[lst[1].tolist(), "fold"] = i_fold

display(train)

In [ ]:
def get_model(model_name, max_length):
    transformer = TFAutoModel.from_pretrained(model_name)
    input_ids = tf.keras.layers.Input(shape=(max_length, ), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_length, ), dtype=tf.int32, name="attention_mask")

    x = transformer(input_ids, attention_mask=attention_mask)
    x = x[0][:, 0, :]

    output = tf.keras.layers.Dense(1, activation="linear")(x)
    model = tf.keras.Model(inputs=[input_ids, attention_mask],
                           outputs=[output])
    return model

def get_tokenizer(model_name):
    return AutoTokenizer.from_pretrained(model_name)

In [ ]:
def get_score(y_true, y_pred):
    """evaluation"""
    return mean_squared_error(y_true, y_pred) ** 0.5


def get_model_inputs(text, tokenizer, max_length):
    """bert model input"""
    input_text = tokenizer.batch_encode_plus(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
    )
    return dict(input_text)


def get_tf_dataset(X, y=None):
    """transform to tf dataset"""
    dataset = (tf.data.Dataset.from_tensor_slices((X, y)).
               batch(Config.batch_size).
               prefetch(tf.data.experimental.AUTOTUNE))
    return dataset


def training(train_df, valid_df, filepath):
    """training for one fold"""
    # model input
    tokenizer = get_tokenizer(model_name=Config.model_name)
    train_x = get_model_inputs(
        text=train_df["excerpt"].tolist(), 
        tokenizer=tokenizer, 
        max_length=Config.max_length)
    
    valid_x = get_model_inputs(
        text=valid_df["excerpt"].tolist(), 
        tokenizer=tokenizer, 
        max_length=Config.max_length)
    
    train_y = train_df[Config.target_col].values
    valid_y = valid_df[Config.target_col].values
    train_dataset = get_tf_dataset(X=train_x, y=train_y)
    valid_dataset = get_tf_dataset(X=valid_x, y=valid_y)
    print("ds",train_dataset)

    # model setting
    model = get_model(model_name=Config.model_name, max_length=Config.max_length)
    optimizer = tfa.optimizers.AdamW(learning_rate=Config.learning_rate, weight_decay=1e-5)
    loss = tf.keras.losses.MeanSquaredError()
    metrics = tf.keras.metrics.RootMeanSquaredError()
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath, 
        monitor="val_loss", 
        verbose=1, 
        save_best_only=True, 
        save_weights_only=True,
        mode="min")
    
    # model training 
    model.fit(
        train_dataset,
        epochs=Config.epochs, 
        verbose=1, 
        callbacks=[checkpoint],
        batch_size=Config.batch_size,
        validation_data=valid_dataset)
    

def inference(test_df, filepath):
    # model inputs
    tokenizer = get_tokenizer(model_name=Config.model_name)
    test_x = get_model_inputs(
        text=test_df["excerpt"].tolist(), 
        tokenizer=tokenizer, 
        max_length=Config.max_length)
    test_dataset = get_tf_dataset(test_x)
    
    # model setting
    model = get_model(model_name=Config.model_name, max_length=Config.max_length)
    model.load_weights(filepath)

    # model prediction
    preds = model.predict(test_dataset)

    return preds.reshape(-1)


def train_cv(train):
    oof = np.zeros(len(train))
    for i_fold in range(Config.n_fold):
        if i_fold in Config.trn_fold:
            K.clear_session()
            filepath = os.path.join(
                        EXP_MODEL,
                        f"{Config.name}-seed{Config.seed}-fold{i_fold}.h5")
            
            valid_mask = np.array(train["fold"] == i_fold, dtype=bool)
            tr_df, va_df = (train[~valid_mask].reset_index(drop=True),
                            train[valid_mask].reset_index(drop=True))
                        
            if not os.path.isfile(filepath):  # if trained model, no training
                training(tr_df, va_df, filepath)
            
            preds = inference(va_df, filepath)
            oof[valid_mask] = preds

            # fold score
            score = get_score(va_df[Config.target_col].values, preds)
            logger.info(f"{Config.name}-seed{Config.seed}-fold{i_fold} >>>>> Score={score:.4f}")

    return oof


def predict_cv(test):
    K.clear_session()
    fold_preds = []
    for i_fold in range(Config.n_fold):
        if i_fold in Config.trn_fold:
            filepath = os.path.join(
                EXP_MODEL,
                f"{Config.name}-seed{Config.seed}-fold{i_fold}.h5")
            
            preds = inference(test, filepath)
            fold_preds.append(preds)
    return fold_preds


def plot_result(target, oof):
    """regression result"""
    fig, ax = plt.subplots(figsize=(8, 8))
    ax = sns.distplot(target, label="y", color='cyan', ax=ax)
    ax = sns.distplot(oof, label="oof", color="magenta", ax=ax)
    ax.legend()
    return fig

In [ ]:
if not Config.only_inference:
    # training
    print("# ---------- # Start Training # ---------- #")
    oof = train_cv(train)
    oof_df = pd.DataFrame(oof, columns=[Config.target_col])
    oof_df.to_csv(os.path.join(EXP_PREDS, "oof_df.csv"), index=False)

    fold_mask = train["fold"].isin(Config.trn_fold)
    target = train[Config.target_col]

    # get oof score
    score = get_score(target[fold_mask].values, oof[fold_mask])
    logger.info(f"OOF-Score >>>>> {score:.4f}")

    # save result
    fig = plot_result(target[fold_mask].values, oof[fold_mask])
    fig.savefig(os.path.join(EXP_FIG, "regression_result.png"), dpi=300)

# prediction
print("# ---------- # Start Inference # ---------- #")
fold_preds = predict_cv(test)
fold_preds_df = pd.DataFrame()
for i in range(len(fold_preds)):  # single fold ok
    fold_preds_df[f"FOLD{i}"] = fold_preds[i]
fold_preds_df.to_csv(os.path.join(EXP_PREDS, f"fold_preds_df.csv"), index=False)

# make submission
print("# ---------- # Make Submission # ---------- #")
sample_submission["target"] = fold_preds_df.mean(axis=1)
filename = Config.name + ".csv" if COLAB else "submission.csv"
sample_submission.to_csv(os.path.join(SUBMISSION, filename), index=False)

In [ ]:
# upload output folder to kaggle dataset
if Config.upload_from_colab:
    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name, upload_dir):
        dataset_metadata = {}
        dataset_metadata['id'] = f'{os.environ["KAGGLE_USERNAME"]}/{dataset_name}'
        dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
        dataset_metadata['title'] = dataset_name
        with open(os.path.join(upload_dir, 'dataset-metadata.json'), 'w') as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode='tar')

    dataset_create_new(dataset_name=EXP, upload_dir=OUTPUT_EXP)